# Distance Metrics and Density Matrices

In this notebook we show how to construct two different kinds of graphs states. The gates we use for entangling qubits in each case are 

1. controlled-Z gates
2. controlled random unitary gates

If the two graph states are constructed from the same graph, then the second can be seen as applying local perturbations to the (controlled) entangling gates to obtain random controlled unitaries. 

In [1]:
import networkx as nx
import cirq

def generate_graph_state(num_qubits):
    # Generate a random graph using NetworkX
    graph = nx.erdos_renyi_graph(num_qubits, 0.5)

    # Initialize a circuit with the specified number of qubits
    circuit = cirq.Circuit()
    qubits = [cirq.GridQubit(i, 0) for i in range(num_qubits)]

    # Apply H gates to all qubits
    circuit.append(cirq.H.on_each(*qubits))

    # Apply CZ gates to pairs of qubits based on the edges of the graph
    for edge in graph.edges:
        qubit1 = qubits[edge[0]]
        qubit2 = qubits[edge[1]]
        circuit.append(cirq.CZ(qubit1, qubit2))

    return circuit


In [2]:
num_qubits = 4
gs1 = generate_graph_state(num_qubits)
gs2 = generate_graph_state(num_qubits)

In [3]:
import numpy as np
import cirq

def trace_distance(circuit1, circuit2):
    # Compute the density matrices of the two circuits
    rho1 = cirq.final_density_matrix(circuit1)
    rho2 = cirq.final_density_matrix(circuit2)

    # Compute the trace distance between the two density matrices
    trace_distance = 0.5 * np.linalg.norm(rho1 - rho2, ord=1)

    return trace_distance


In [4]:
gs1

┌──┐
(0, 0): ───H────@─────────────
                │
(1, 0): ───H────┼@────@───────
                ││    │
(2, 0): ───H────┼@────┼───@───
                │     │   │
(3, 0): ───H────@─────@───@───
               └──┘

In [5]:
gs2

(0, 0): ───H───@───@───────────────
               │   │
(1, 0): ───H───@───┼───@───@───────
                   │   │   │
(2, 0): ───H───────@───@───┼───@───
                           │   │
(3, 0): ───H───────────────@───@───

In [6]:
trace_distance(gs1, gs2)

0.7499999403953552

In [7]:
import cirq
import numpy as np

def graph_state_circuit(num_qubits):
    """Returns a graph state circuit on num_qubits qubits"""
    # Generate a random graph with num_qubits nodes
    graph = nx.fast_gnp_random_graph(num_qubits, 0.5)

    # Initialize the circuit with the given qubits
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
    circuit = cirq.Circuit()

    # Apply Hadamard gates to all qubits
    circuit.append(cirq.H.on_each(qubits))

    # Apply random controlled unitary gates to each edge
    for edge in graph.edges():
        control_qubit, target_qubit = qubits[edge[0]], qubits[edge[1]]

        # Generate a random unitary matrix
        random_unitary = cirq.testing.random_unitary(2)

        # Apply the controlled random unitary gate to the circuit
        circuit.append(cirq.ControlledGate(cirq.MatrixGate(random_unitary)).on(control_qubit, target_qubit))

        
    # Define number of qubits in circuit
    circuit.num_qubits = len(circuit.all_qubits())

    
    return circuit

In [8]:
circuit = graph_state_circuit(4)
circuit

0: ───H───@───────────────────────────────@───────────────────────────────@─────────────────────────────────────────────────────────────
          │                               │                               │
          ┌                           ┐   │                               │
1: ───H───│-0.687-0.375j  0.176-0.597j│───┼───────────────────────────────┼───────────────────────────────@─────────────────────────────
          │-0.608-0.132j -0.001+0.783j│   │                               │                               │
          └                           ┘   │                               │                               │
                                          │                               │                               │
                                          ┌                           ┐   │                               │
2: ───H───────────────────────────────────│-0.53 -0.7j   -0.451+0.161j│───┼───────────────────────────────┼─────────────────────────────
                                          │-0.462-0.123j  0.471-0.741j│   │                               │
                                          └                           ┘   │                               │
                                                                          │                               │
                                                                          ┌                           ┐   ┌                         ┐
3: ───H───────────────────────────────────────────────────────────────────│ 0.092+0.159j  0.969-0.166j│───│0.337-0.811j 0.313+0.36j │───
                                                                          │-0.603+0.776j -0.097-0.156j│   │0.466+0.103j 0.552-0.684j│
                                                                          └                           ┘   └                         ┘

In [9]:
trace_distance(gs1, circuit)

0.7174839377403259

## Generate Two Graph States with the Same Graph

In [10]:
num_qubits = 5
graph1 = nx.erdos_renyi_graph(num_qubits, 0.5)

In [11]:
def generate_graph_state(graph):
    # Generate a random graph using NetworkX
    graph = graph
    num_qubits = len(graph.nodes)
    # Initialize a circuit with the specified number of qubits
    circuit = cirq.Circuit()
    qubits = [cirq.GridQubit(i, 0) for i in range(num_qubits)]

    # Apply H gates to all qubits
    circuit.append(cirq.H.on_each(*qubits))

    # Apply CZ gates to pairs of qubits based on the edges of the graph
    for edge in graph.edges:
        qubit1 = qubits[edge[0]]
        qubit2 = qubits[edge[1]]
        circuit.append(cirq.CZ(qubit1, qubit2))

    return circuit

In [12]:
GS1 = generate_graph_state(graph1)
GS1

┌──┐
(0, 0): ───H───@───@────@─────────────
               │   │    │
(1, 0): ───H───┼───┼────┼─────────────
               │   │    │
(2, 0): ───H───@───┼────┼@────@───────
                   │    ││    │
(3, 0): ───H───────@────┼@────┼───@───
                        │     │   │
(4, 0): ───H────────────@─────@───@───
                       └──┘

The following generates random controlled unitaries to entangle qubits. Each time the functions is called it generates new random controlled unitaries. 

In [13]:
import cirq
import numpy as np

def graph_state_circuit(graph):
    """Returns a graph state circuit on num_qubits qubits"""
    # Generate a random graph with num_qubits nodes
    graph = graph
    num_qubits = len(graph.nodes)

    # Initialize the circuit with the given qubits
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
    circuit = cirq.Circuit()

    # Apply Hadamard gates to all qubits
    circuit.append(cirq.H.on_each(qubits))

    # Apply random controlled unitary gates to each edge
    for edge in graph.edges():
        control_qubit, target_qubit = qubits[edge[0]], qubits[edge[1]]

        # Generate a random unitary matrix
        random_unitary = cirq.testing.random_unitary(2)

        # Apply the controlled random unitary gate to the circuit
        circuit.append(cirq.ControlledGate(cirq.MatrixGate(random_unitary)).on(control_qubit, target_qubit))

        
    # Define number of qubits in circuit
    circuit.num_qubits = len(circuit.all_qubits())

    
    return circuit

In [14]:
GS2 = graph_state_circuit(graph1)
GS2

┌──────────────────────────────────────────────────────────┐
0: ───H───@───────────────────────────────@────────────────────────────────@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          │                               │                                │
1: ───H───┼───────────────────────────────┼────────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          │                               │                                │
          ┌                           ┐   │                                │
2: ───H───│ 0.059+0.493j  0.696+0.518j│───┼────────────────────────────────┼────────────────────────────@────────────────────────────────@───────────────────────────────────────────────────────────────
          │ 0.631+0.596j -0.496+0.027j│   │                                │                            │                                │
          └                           ┘   │                                │                            │                                │
                                          │                                │                            │                                │
                                          ┌                           ┐    │                            ┌                           ┐    │
3: ───H───────────────────────────────────│-0.371+0.73j   0.088+0.567j│────┼────────────────────────────│ 0.242+0.217j -0.234+0.916j│────┼───────────────────────────────@───────────────────────────────
                                          │ 0.449-0.358j -0.221+0.789j│    │                            │-0.865-0.381j  0.022+0.325j│    │                               │
                                          └                           ┘    │                            └                           ┘    │                               │
                                                                           │                                                             │                               │
                                                                           ┌                           ┐                                 ┌                           ┐   ┌                           ┐
4: ───H────────────────────────────────────────────────────────────────────│-0.834-0.135j -0.381+0.376j│─────────────────────────────────│-0.597+0.51j   0.583-0.206j│───│ 0.661+0.387j -0.302-0.567j│───
                                                                           │-0.442+0.302j  0.028-0.844j│                                 │-0.523-0.33j  -0.47 -0.63j │   │ 0.213-0.607j  0.595-0.483j│
                                                                           └                           ┘                                 └                           ┘   └                           ┘
                                                                          └──────────────────────────────────────────────────────────┘

In [15]:
trace_distance(GS1, GS2)

1.06827974319458